# Setup

In [1]:
%load_ext autoreload
%autoreload 2
%aimport

Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
%matplotlib inline 

In [3]:
import matplotlib.pyplot as plt
import torch

from bliss.inference import SDSSFrame
from bliss.datasets import sdss
from bliss.inference import reconstruct_scene_at_coordinates
from case_studies.sdss_galaxies.plots import load_models


from astropy.table import Table

import plotly.express as px
import plotly.graph_objects as go

Global seed set to 40


In [4]:
# check GPU is configured correctly
device = torch.device('cuda:0')
!echo $CUDA_VISIBLE_DEVICES

5


# Get Config

In [5]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])
    

# Run

In [9]:
from bliss.datasets.galsim_galaxies import SingleGalsimGalaxies

In [10]:
ds = instantiate(cfg.datasets.sdss_galaxies)

In [14]:
dl = ds.val_dataloader()
for x in dl:
    print(x['images'].shape)

Generating fixed validation set: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.50it/s]

torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])


In [15]:
for x in dl:
    print(x['images'].shape)

torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])
torch.Size([64, 1, 53, 53])


In [ ]:
# Load data

# load sdss data
sdss_dir = '/home/imendoza/bliss/data/sdss/'
pixel_scale = 0.393
coadd_file = "/home/imendoza/bliss/data/coadd_catalog_94_1_12.fits"
frame = SDSSFrame(sdss_dir, pixel_scale, coadd_file)

In [ ]:
# load models
from hydra import compose, initialize
from hydra.utils import instantiate
from bliss.encoder import Encoder

with initialize(config_path="../../config"):
    cfg = compose("config", overrides=[])
    
    
enc, dec = load_models(cfg, device)
bp = enc.border_padding
torch.cuda.empty_cache()